In [20]:
# loader = CSVLoader(file_path="C:/Users/Deepin/Desktop/shivam/RAG Projects/RAG Projects/data/test.csv")
# loader = TextLoader(file_path="C:/Users/Deepin/Desktop/shivam/RAG Projects/RAG Projects/data/test.txt")
# loader = PyMuPDFLoader(file_path="C:/Users/Deepin/Desktop/shivam/RAG Projects/RAG Projects/data/test.pdf")
# loader = Docx2txtLoader("data/test.docx")
# loader = UnstructuredURLLoader(urls = ['https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture01.pdf'])
# loader = WebBaseLoader("https://medium.com/@varsha.rainer/document-loaders-in-langchain-7c2db9851123")
# loader.load()
# pagee = pages[0]
# print(pagee.page_content[:500])


# **Basic chat bot**

In [2]:
from langchain.embeddings import OpenAIEmbeddings, CohereEmbeddings
from langchain.vectorstores import Chroma
import streamlit as st
import os
import glob
from typing import Union
from io import BytesIO
from typing import List
from dotenv import load_dotenv
from multiprocessing import Pool
from constants import CHROMA_SETTINGS
import tempfile
from tqdm import tqdm
import argparse
import time
from PIL import Image
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.docstore.document import Document
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS,Chroma
from langchain_community.llms import Ollama
from langchain_cohere import CohereEmbeddings

load_dotenv()
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import TextLoader, PyMuPDFLoader, Docx2txtLoader, UnstructuredURLLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

def get_text_chunks(results, chunk_size, chunk_overlap):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    texts = text_splitter.split_documents(results)
    return texts 

def get_vectorstore(results, embeddings_model_name, persist_directory, client_settings, chunk_size, chunk_overlap):
    if embeddings_model_name == "openai":     
        embeddings = OpenAIEmbeddings()
        print('OpenAI embeddings loaded')
    elif embeddings_model_name == "Cohereembeddings":
        embeddings = CohereEmbeddings()
        print('Cohere embeddings loaded')
    else:
        raise ValueError("Unsupported embeddings model name.")
       
    texts = get_text_chunks(results, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    
    db = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory, client_settings=client_settings)
    db.add_documents(texts)
    return db

def get_conversation_chain(vectorstore, target_source_chunks, model_type):
    retriever = vectorstore.as_retriever(search_kwargs={"k": target_source_chunks})
    
    match model_type:
        case "OpenaAI":
            llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
        case "Llama3":
            llm = Ollama(model="llama3")
        case _default:
            raise Exception(f"Model type {model_type} is not supported. Please choose one of the following: OpenaAI, Llama3")
    
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory
    )
    return conversation_chain

def load_input_file(input_file):
    extension = input_file.split(".")[-1]
    first_extension = input_file.split(":")[0]

    if first_extension.lower() in ["http", "https"]:
        if extension == "pdf":
            loader = UnstructuredURLLoader(urls=[input_file])
        else:
            loader = WebBaseLoader(input_file)
    else:
        match extension:
            case "txt":
                loader = TextLoader(file_path=input_file)
            case "pdf":
                loader = PyMuPDFLoader(file_path=input_file)
            case "docx":
                loader = Docx2txtLoader(file_path=input_file)
            case _:
                raise ValueError(f"Unsupported file extension: {extension}")
    
    return loader.load()

# Main flow
input_file = input("Enter file name: ")
raw_text = load_input_file(input_file)

chunk_size = 1000  # Adjust as needed
chunk_overlap = 200  # Adjust as needed
embeddings_model_name = os.environ.get('EMBEDDINGS_MODEL_NAME')
persist_directory = os.environ.get('PERSIST_DIRECTORY')
client_settings = {}  # Replace with actual client settings if needed
target_source_chunks = 5  # Adjust as needed
model_type=os.environ.get('MODEL_TYPE')

text_chunks = get_text_chunks(results=raw_text, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
vectorstore = get_vectorstore(results=text_chunks, embeddings_model_name=embeddings_model_name, persist_directory=persist_directory, client_settings=client_settings, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
conversation_chain = get_conversation_chain(vectorstore=vectorstore, target_source_chunks=target_source_chunks, model_type=model_type)

# Now you can use the conversation_chain to ask questions
question = input("Enter your question: ")
response = conversation_chain(question)
print("Response:", response)


Cohere embeddings loaded
Response: {'question': 'hi', 'chat_history': [HumanMessage(content='hi'), AIMessage(content="I don't know.")], 'answer': "I don't know."}


# **Langchain Langsmith Langserve**

In [ ]:
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import JSONResponse
from langchain.embeddings import OpenAIEmbeddings, CohereEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOpenAI
from langchain_community.llms import Ollama
from langchain.document_loaders import TextLoader, PyMuPDFLoader, Docx2txtLoader, UnstructuredURLLoader, WebBaseLoader
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

# Define constants
CHROMA_SETTINGS = {}  # Replace with your actual settings if needed

app = FastAPI()

def get_text_chunks(results, chunk_size, chunk_overlap):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    texts = text_splitter.split_documents(results)
    return texts 

def get_vectorstore(results, embeddings_model_name, persist_directory, client_settings, chunk_size, chunk_overlap):
    if embeddings_model_name == "openai":     
        embeddings = OpenAIEmbeddings()
        print('OpenAI embeddings loaded')
    elif embeddings_model_name == "Cohereembeddings":
        embeddings = CohereEmbeddings()
        print('Cohere embeddings loaded')
    else:
        print(f"Embeddings model name provided: {embeddings_model_name}")  # Debug statement
        raise ValueError("Unsupported embeddings model name.")
       
    texts = get_text_chunks(results, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    db = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory, client_settings=client_settings)
    db.add_documents(texts)
    return db

def get_conversation_chain(vectorstore, target_source_chunks, model_type):
    retriever = vectorstore.as_retriever(search_kwargs={"k": target_source_chunks})
    
    if model_type == "OpenAI":
        llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    elif model_type == "Llama3":
        llm = Ollama(model="llama3")
    else:
        raise Exception(f"Model type {model_type} is not supported. Please choose one of the following: OpenAI, Llama3")
    
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory
    )
    return conversation_chain

def load_input_file(file_path):
    extension = file_path.split(".")[-1]
    first_extension = file_path.split(":")[0]

    if first_extension.lower() in ["http", "https"]:
        if extension == "pdf":
            loader = UnstructuredURLLoader(urls=[file_path])
        else:
            loader = WebBaseLoader(file_path)
    else:
        if extension == "txt":
            loader = TextLoader(file_path=file_path)
        elif extension == "pdf":
            loader = PyMuPDFLoader(file_path=file_path)
        elif extension == "docx":
            loader = Docx2txtLoader(file_path=file_path)
        else:
            raise ValueError(f"Unsupported file extension: {extension}")
    
    return loader.load()

@app.post("/process_file/")
async def process_file(file: UploadFile = File(...), model_type: str = Form(...)):
    temp_dir = "temp"
    os.makedirs(temp_dir, exist_ok=True)
    file_location = f"{temp_dir}/{file.filename}"
    
    with open(file_location, "wb+") as file_object:
        file_object.write(file.file.read())

    raw_text = load_input_file(file_location)
    chunk_size = 1000  # Adjust as needed
    chunk_overlap = 200  # Adjust as needed
    embeddings_model_name = os.getenv('EMBEDDINGS_MODEL_NAME')
    persist_directory = os.getenv('PERSIST_DIRECTORY')
    client_settings = CHROMA_SETTINGS  # Replace with actual client settings if needed
    target_source_chunks = 5  # Adjust as needed

    # Debug statements
    print(f"Embeddings model name: {embeddings_model_name}")
    print(f"Persist directory: {persist_directory}")
    print(f"Model type: {model_type}")

    text_chunks = get_text_chunks(results=raw_text, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    vectorstore = get_vectorstore(results=text_chunks, embeddings_model_name=embeddings_model_name, persist_directory=persist_directory, client_settings=client_settings, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    global conversation_chain
    conversation_chain = get_conversation_chain(vectorstore=vectorstore, target_source_chunks=target_source_chunks, model_type=model_type)

    return JSONResponse(content={"message": "File processed successfully", "model_type": model_type})

@app.post("/ask_question/")
async def ask_question(question: str = Form(...)):
    global conversation_chain
    response = conversation_chain(question)

    # Debugging the response
    print(f"Response: {response}")

    if isinstance(response, dict) and 'result' in response:
        response_text = response['result']
    else:
        response_text = str(response)
    
    return JSONResponse(content={"response": response_text})

# To run the FastAPI app:
# uvicorn main:app --reload
